### GOAL
Here I explore training results/validation results for a few algorithms, trying to predict DOC with one two preprocessing pipelines designed in 
the notebook "DOC_5_Pipeline_Design_for_PreProcessing.ipynb"

In [98]:
import pandas as pd
import numpy as np
import pickle
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.tree import DecisionTreeRegressor, export_graphviz
import graphviz
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.externals import joblib
from seaborn import heatmap
from IPython.core.display import HTML, display

In [60]:
from sklearn import __version__ as skl_version

In [61]:
print(skl_version)

0.19.0


In [2]:
%matplotlib inline
display(HTML("<style>.container {width: 90% !important}</style>"))

In [47]:
def GetRrsIdx(df, label_list=None):
    if label_list:
        return [df.columns.get_loc(label) for label in label_list]
    else:
        return [df.columns.get_loc(col) for col in df.filter(like='Rrs', axis=1).columns]


class BandRatioAdder(BaseEstimator, TransformerMixin):
    """ class to add attributes. """
    def __init__(self):
        return None
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        band_ratio_531_443 = X[:, rrs531_ix] / X[:, rrs443_ix]
        band_ratio_555_443 = X[:, rrs555_ix] / X[:, rrs443_ix]
        return np.c_[X, band_ratio_531_443, band_ratio_555_443]


class RrsLogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        
        X[:, rrs_ix_list] = np.log(X[:, rrs_ix_list])
        return X

def AssessModel(regressor, feats, labels, model_name=''):
    preds = regressor.predict(feats)
    model_mse = mean_squared_error(labels, preds)
    print("%s rmse: %.3f" % (model_name, np.sqrt(model_mse)))

def display_scores(scores):
    rmse_scores = np.sqrt(-scores)
    print("Scores: " , rmse_scores)
    print("Mean: ", rmse_scores.mean())
    print("Standard deviation", rmse_scores.std())

doc_ag412_pipeline = Pipeline([('imputer', Imputer(strategy='median')),
                               ('br_adder', BandRatioAdder()),
                               ('std_scaler', StandardScaler())])

In [4]:
with open('./PklJar/TrainSet.pkl', 'rb') as fb:
    trainFrames = pickle.load(fb)
dfTrainFeatures = trainFrames['features']
dfTrainLabels = trainFrames['labels']

**CAUTION:** functions implemented in the pipeline must be in the namespace.

In this case, these are contained in Helpers.py (imported above)

In [5]:
rrs443_ix, rrs531_ix, rrs555_ix = GetRrsIdx(dfTrainFeatures,
                                            label_list=['Rrs443', 'Rrs531', 'Rrs555'])
rrs_ix_list = GetRrsIdx(dfTrainFeatures)
rrs_ix_list = rrs_ix_list + [rrs_ix_list[-1] + 1, rrs_ix_list[-1] + 2]

Training & evaluation on the Training Set

In [6]:
doc_features_preprocessed = doc_ag412_pipeline.fit_transform(dfTrainFeatures.values)

In [7]:
doc_labels = dfTrainLabels.as_matrix(columns=['doc'])

In [8]:
lin_reg_doc = LinearRegression()
lin_reg_doc.fit(doc_features_preprocessed, doc_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [30]:
AssessModel(lin_reg_doc, doc_features_preprocessed, doc_labels, model_name='lin_reg' )

lin_reg rmse: 21.584


In [34]:
sgd_doc = SGDRegressor(max_iter=1000)
sgd_doc.fit(doc_features_preprocessed, doc_labels.reshape((-1,)))

SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=1000, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False)

In [35]:
AssessModel(sgd_doc, doc_features_preprocessed, doc_labels, model_name='sgd')

sgd rmse: 21.601


In [36]:
tree_doc = DecisionTreeRegressor()
tree_doc.fit(doc_features_preprocessed, doc_labels)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [37]:
AssessModel(tree_doc, doc_features_preprocessed, doc_labels, model_name='d-tree')

d-tree rmse: 0.000


Mitigating the decision tree overfit using k-fold cross-validation

In [46]:
tree_scores = cross_val_score(tree_doc, doc_features_preprocessed, doc_labels,
                        scoring='neg_mean_squared_error', cv=10)

In [48]:
display_scores(tree_scores)

Scores:  [ 27.35474036  31.00433296  16.81202746  25.80543546  20.4533907
  25.46563619  36.15110518  42.27280109  32.35785155  30.10684743]
Mean:  28.7784168402
Standard deviation 7.00233708537


In [44]:
# comparing to lin_reg and sgd
lin_reg_scores = cross_val_score(lin_reg_doc, doc_features_preprocessed, doc_labels,
                                scoring='neg_mean_squared_error', cv=10)
sgd_scores = cross_val_score(sgd_doc, doc_features_preprocessed, doc_labels.reshape((-1,)),
                            scoring='neg_mean_squared_error', cv=10)

In [49]:
display_scores(lin_reg_scores)

Scores:  [ 20.73787638  20.15781999  21.87362437  15.42237922  17.06222598
  28.24074133  28.27423931  42.74516779  22.1058328   21.69912084]
Mean:  23.8319028007
Standard deviation 7.40180538491


In [50]:
display_scores(sgd_scores)

Scores:  [ 20.88940831  20.22516235  21.73665881  15.24791377  16.48080812
  27.34215976  28.45216958  42.36234314  22.00228579  21.64682123]
Mean:  23.6385730858
Standard deviation 7.33876462849


In [64]:
features_names = dfTrainFeatures.columns.tolist() + ['br_531_443', 'br_555_443']

In [87]:
feature_scores = [(name,score) for name, score in zip(features_names,
                                                     tree_doc.feature_importances_)]

In [88]:
feature_scores

[('SST', 0.023060026520039169),
 ('SSS', 0.86001633157142854),
 ('Rrs412', 0.0073552344742274459),
 ('Rrs443', 0.00087997498170493474),
 ('Rrs531', 0.028769476761792284),
 ('Rrs555', 0.021440038049392696),
 ('Rrs667', 0.015271590344997988),
 ('br_531_443', 0.033028757552767518),
 ('br_555_443', 0.010178569743649491)]

In [91]:
sorted_feature_scores = sorted(feature_scores, key=lambda x: x[1], reverse=True)

In [92]:
sorted_feature_scores

[('SSS', 0.86001633157142854),
 ('br_531_443', 0.033028757552767518),
 ('Rrs531', 0.028769476761792284),
 ('SST', 0.023060026520039169),
 ('Rrs555', 0.021440038049392696),
 ('Rrs667', 0.015271590344997988),
 ('br_555_443', 0.010178569743649491),
 ('Rrs412', 0.0073552344742274459),
 ('Rrs443', 0.00087997498170493474)]

In [ ]:
graphviz.Source(export_graphviz(tree_doc, out_file=None, feature_names=features_names))

In [94]:
forest_doc = RandomForestRegressor()
forest_doc.fit(doc_features_preprocessed, doc_labels.ravel())

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [96]:
forest_scores = cross_val_score(forest_doc, doc_features_preprocessed,
                                doc_labels.ravel(), 
                                scoring='neg_mean_squared_error', cv=10)

In [97]:
display_scores(forest_scores)

Scores:  [ 17.24582528  20.42944063  18.27214692  14.75014539  16.02748236
  21.93918614  29.58377162  31.15810535  17.67575827  24.96909436]
Mean:  21.2050956332
Standard deviation 5.38313220924


**Fine Tuning HyperParameters** of the random forest regressor with:
* scikit-learn's GridSearchCV
* scikit-optimize's BayesSearchCV

In [99]:
# First pickle all models trained so far...
joblib.dump(lin_reg_doc, './PklJar/Models/doc_6_lin_reg.pkl')
joblib.dump(sgd_doc, './PklJar/Models/doc_6_sgd.pkl')
joblib.dump(tree_doc, './PklJar/Models/doc_6_tree.pkl')
joblib.dump(forest_doc, './PklJar/Models/doc_6_forest.pkl')

['./PklJar/Models/doc_6_forest.pkl']